In [ ]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))

In [ ]:
# === Ouroboros Recommendation Pipeline ===
#
# 1. Create or retrieve dataset embedding
# 2. Get user features, social graph, and engagement history
# 3. Classify in-network and out-network posts (TF-IDF + cosine similarity)
# 4. Hydrate retrieved posts with full content
# 5. Pre-filtering (user guardrails: blocked owners, tags, disliked posts)
# 6. Scoring (engagement + relevance + recency + tag overlap + network weight)
# 7. Diversity injection (prevent filter bubbles)
# 8. Post-filtering (social guardrails: PG restrictions, high-dislike suppression)
# 9. Cold-start fallback for new users
# 10. Display ranked feed + record engagement feedback loop

In [ ]:
from datasets.retrieve import Embedding
from context.user.user import User
from core.utils import filter, rank
from core.models import Engine

# ── 1. Create or retrieve dataset embedding ──
embed = Embedding()
print(f"Loaded {len(embed.get_innetwork_posts())} in-network, {len(embed.get_outnetwork_posts())} out-network posts")

# ── 2. Get user features and actions ──
user = User()
user_state = user.embedding()
print(f"User: {user_state.profile.name} | Following: {user_state.socialgraph.following.len} | Friends: {user_state.socialgraph.friends}")
print(f"Interest tags: {user_state.profile.interest_tags}")
print(f"Cold start: {user.is_cold_start}")

# ── 3. Classify in-network and out-network posts (embed + score) ──
if user.is_cold_start:
    # cold-start fallback: rank by raw engagement + recency, no personalization
    print("\n⚠ Cold-start user — using fallback ranking")
    all_posts = embed.get()
    scored_combined = Engine.cold_start_fallback(all_posts)
else:
    scored_in, scored_out = embed.classify(user_state)
    print(f"\nScored in-network: {len(scored_in)} posts")
    for sp in scored_in:
        print(f"  [{sp.post.id}] {sp.post.title:<25} score={sp.score:.4f}  relevance={sp.relevance:.3f}  engagement={sp.engagement:.1f}  recency={sp.recency:.3f}")
    print(f"\nScored out-network: {len(scored_out)} posts")
    for sp in scored_out:
        print(f"  [{sp.post.id}] {sp.post.title:<25} score={sp.score:.4f}  relevance={sp.relevance:.3f}  engagement={sp.engagement:.1f}  recency={sp.recency:.3f}")
    scored_combined = scored_in + scored_out

# ── 4. Hydrate retrieved posts ──
# posts are already hydrated from JSON; in production this would fetch attachments/CDN links
hydrated = [sp for sp in scored_combined]  # already full objects
print(f"\nHydrated {len(hydrated)} total posts")

# ── 5. Pre-filtering (user guardrails) ──
prefiltered = user.filter(hydrated)
removed = len(hydrated) - len(prefiltered)
print(f"Pre-filter: {removed} posts removed (blocked owners/tags/disliked) → {len(prefiltered)} remaining")

# ── 6. Engagement-based ranking (normalize scores) ──
ranked = rank(prefiltered)

# ── 7. Diversity injection ──
diverse_ranked = Engine.rank_with_diversity(ranked)
print(f"\nDiversity re-ranking applied to {len(diverse_ranked)} posts")

# ── 8. Post-filtering (social guardrails) ──
final_feed = filter(diverse_ranked, user_state)
removed_post = len(diverse_ranked) - len(final_feed)
print(f"Post-filter: {removed_post} posts removed (PG/social guardrails) → {len(final_feed)} remaining")

In [ ]:
# ── 9. Final ranked feed ──
print("=" * 80)
print(f"{'RANK':<5} {'ID':<12} {'TITLE':<28} {'OWNER':<15} {'SOURCE':<12} {'SCORE':<8} {'DIV':<8}")
print("-" * 80)
for i, sp in enumerate(final_feed, 1):
    print(f"{i:<5} {sp.post.id:<12} {sp.post.title:<28} {sp.post.owner:<15} {sp.source:<12} {sp.score:<8.4f} {sp.diversity_bonus:<+8.4f}")
print("=" * 80)
print(f"\nFeed size: {len(final_feed)} posts from {len(set(sp.post.owner for sp in final_feed))} unique authors")

In [ ]:
# ── 10. Engagement feedback loop (simulate) ──
# in production: UI sends back engagement events in real-time
engagement_params = {
    "events": [
        {"post_id": "post_004", "action": "click", "dwell_seconds": 30.0},
        {"post_id": "post_101", "action": "like", "dwell_seconds": 10.0},
        {"post_id": "post_103", "action": "dislike", "dwell_seconds": 2.0},
    ],
    "block_owners": [],
    "block_tags": [],
}

user.update_filters(engagement_params)
print("Engagement recorded:")
for evt in engagement_params["events"]:
    print(f"  {evt['action']:<8} on {evt['post_id']} ({evt['dwell_seconds']}s dwell)")
print(f"\nUpdated disliked posts: {user.tree.anti_filters.disliked_posts}")
print(f"Total engagement history: {len(user.tree.anti_filters.engagement_history)} events")